<a href="https://colab.research.google.com/github/Ismail-Armutcu/Algorithms-for-Interactive-Sytems/blob/main/MMI513_PA6_BeeDroids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [23]:
DAVOIDANCE = 3
DATTRACTANCE = 1
ATTRACTANCETIME = 3

def unitvector(vec):
    nrm =  np.linalg.norm(vec)
    if nrm == 0:
        vec = np.random.rand(2)
    return vec / np.linalg.norm(vec)

def initialpositions(numboids, xlims, ylims, obstacleset):
    count = 0
    post = np.empty((1,2))
    while count < numboids:
        x = np.random.rand(1) * (xlims[1] - xlims[0]) + xlims[0]
        y = np.random.rand(1) * (ylims[1] - ylims[0]) + ylims[0]
        pos = np.array([x, y]).reshape((1,2))
        inflag = False
        for obstacle in obstacleset:
            posobs = obstacle.getposition()
            dist = np.linalg.norm(pos - posobs)
            if dist < obstacle.getradius():
                inflag = True
        if not inflag:
            post = np.concatenate([post, pos], axis=0)
            count += 1
    return post

def initialheadings(numboids):
    thetas = np.random.rand(numboids) * 2 * np.pi
    heads = np.zeros((numboids, 2))
    for ind in range(numboids):
        heads[ind, :] = np.array([np.cos(thetas[ind]), np.sin(thetas[ind])])
    return heads

def initialspeeds(numboids, minspeed, maxspeed):
    speeds = np.random.rand(numboids) * (maxspeed - minspeed) + minspeed
    return speeds

def cart2polar(xvec):
    th = np.arctan2(xvec[1], xvec[0])
    return th

def setdeepcopy(A):
    B = set()
    for item in A:
        B.add(item.copy())
    return B

def setcombine(A, B):
    C = setdeepcopy(A)
    if B==None:
        return A
    for item in B:
        C.update({item.copy()})
    return C

In [24]:
# @title
class Boid(object):
    def __init__(self, position = np.array([0., 0.]), heading = np.array([1., 1.]), leadflag = False, perceptionradius = 2., speed = 1, id = 0):
        super().__init__()
        self._position = position
        self._heading = unitvector(heading)
        self._leadflag = leadflag
        self._perceptionradius = perceptionradius
        self._velocity = self._heading * speed
        self._id = id
    def getid(self):
        return self._id
    def getposition(self):
        return self._position

    def setposition(self, position):
        self._position = position

    def getheading(self):
        return self._heading

    def setheading(self, heading):
        self._heading = unitvector(heading)

    def getleader(self):
        return self._leadflag

    def setleader(self, leadflag):
        self._leadflag = leadflag

    def getradius(self):
        return self._perceptionradius

    def setradius(self, perceptionradius):
        self._perceptionradius = perceptionradius

    def getvelocity(self):
        return self._velocity

    def setvelocity(self, speed):
        self._velocity = unitvector(self._velocity) * speed
        self.setheading(unitvector(self._velocity))

    def addvelocity(self, velocity):
        self._velocity += velocity

    def copy(self):
        newboid = Boid()
        newboid.setheading(self._heading)
        newboid.setleader(self._leadflag)
        newboid.setposition(self._position)
        return newboid

    def vectortoother(self, otherboid):
        return self._position - otherboid.getposition()

    def distancetoother(self, other):
        v = self.vectortoother(other)
        return np.linalg.norm(v)

    def othervisible(self, otherboid):
        dst = self.distancetoother(otherboid)
        if dst < self._perceptionradius:
            return True
        return False

    def move(self):
        self._position += self._velocity





TypeError: Attractor.__init__() takes from 1 to 3 positional arguments but 4 were given

In [ ]:
class Obstacle(object):

    def __init__(self, position = np.array([0., 0.]), radiusbb = 0.3):
        super().__init__()
        self._position = position
        self._radiusbb = radiusbb

    def getposition(self):
        return self._position

    def setposition(self, position):
        self._position = position

    def getradius(self):
        return self._radiusbb

    def setradius(self, radiusbb):
        self._radiusbb = radiusbb

    def copy(self):
        newobstacle = Obstacle()
        newobstacle.setposition(self._position)
        newobstacle.setradius(self._radiusbb)
        return newobstacle

In [ ]:

class Attractor(object):

    def __init__(self, position = np.array([0., 0.]), radius = 0.3):
        super().__init__()
        self._time = 0
        self._position = position
        self._radius = radius
        self._active = True
        self._visitedBoids = []
        self._visitors = {}  # Dictionary to track bees and their visiting durations

    def getactive(self):
      return self._active

    def setactive(self,active):
        self._active = active

    def getposition(self):
        return self._position

    def setposition(self, position):
        self._position = position

    def getradius(self):
        return self._radius

    def setradius(self, radius):
        self._radius = radius

    def isactiveforboid(self, boid):
        if self._active == False:
          return False
        elif len(self._visitors) >= 3:
          return False
        elif boid.getid() in self._visitors: ## Burası değişebilir
          return True
        elif boid.getid() not in self._visitedBoids:
          return False
        else:
          return True

    def visit(self, boid):
        if self._active == False:
          print("Attractor::visit aktif değilken visit edemezsin")
        elif boid.getid() not in self._visitedBoids:
          self._visitors[boid.getid()] = self._time  # Start timing the visit duration
          self._visitedBoids.append(boid.getid()) ## Burda mı append etmeliyim yoka updatede visitordan silerkenmi
        else:
          print("Boid: ",boid.getid(),"has visited before")

    def update(self):
        self._time += 1

        for boid in self._visitors:
            visitTime = self._time-self._visitors[boid]
            if visitTime >= ATTRACTANCETIME:
                del self._visitors[boid]

        if len(self._visitors) == 3:
          self._active = False
        else:
          self._active = True
    def copy(self):
        newattractor = Attractor()
        newattractor.setposition(self._position)
        newattractor.setradius(self._radius)
        newattractor.setactive(self._active)
        newattractor._visitedBoids = self._visitedBoids
        newattractor._visitors = self._visitors
        newattractor._time = self._time
        return newattractor


In [ ]:
def flockset(numboids, xlims, ylims, minspeed, maxspeed, obstacleset, leaderind):
    F = set()
    post = initialpositions(numboids, xlims, ylims, obstacleset)
    heads = initialheadings(numboids)
    speeds = initialspeeds(numboids, minspeed, maxspeed)
    for ind in range(numboids):
        if ind == leaderind:
            F.add(Boid(position=post[ind,:], heading=heads[ind,:], leadflag=False, perceptionradius = 3, speed=speeds[ind], id = ind))
        else:
            F.add(Boid(position=post[ind,:], heading=heads[ind,:], leadflag=False,perceptionradius = 3, speed=speeds[ind], id = ind))
    return F

def obstacleset(positions, radii):
    Oset = set()
    for ind in range(len(positions)):
        obs = Obstacle(positions[ind], radii[ind])
        Oset.add(obs)
    return Oset

def attractorset(positions, radii):
    Aset = set()
    for ind in range(len(positions)):
        attr = Attractor(positions[ind], radii[ind])
        Aset.add(attr)
    return Aset


In [ ]:
def plotflockset(F, fig,ax):
    xs, ys, ths = [], [], []
    for boid in F:
        position = boid.getposition()
        heading = boid.getheading()
        x, y = position[0], position[1]
        th = cart2polar(heading)
        ths.append(np.rad2deg(th))
        xs.append(x)
        ys.append(y)
        ax.scatter(x, y, marker=(3, 0, np.rad2deg(th)))

    fig.tight_layout()
    return fig,ax

def plotobstacleset(A,fig,ax):
    for obs in A:
        [x,y] = obs.getposition()
        ax.scatter(x,y,marker ='o')
        #circle = plt.Circle(obs.getposition(), obs.getradius())
        #ax.add_patch(circle)
    #fig.tight_layout()
    return fig,ax

def plotattractorset(AttractorSet,fig,ax):

    for attr in AttractorSet:
      [x,y] = attr.getposition()
      ax.scatter(x,y,marker ='*')
    fig.tight_layout()
    return fig,ax


In [ ]:
def separation(M, boid, ds = 0.3):
    mindist = np.Inf
    cboid = Boid()
    for other in M:
        d = boid.distancetoother(other)
        if d < mindist:
            mindist = d
            cboid = other
    v = cboid.getposition() - boid.getposition()
    r = 1 - 2 * ds / (np.linalg.norm(v) + ds)
    return r * unitvector(v)

def alignment(M, boid):
    mindist = np.Inf
    cboid = Boid()
    for other in M:
        d = boid.distancetoother(other)
        if d < mindist:
            mindist = d
            cboid = other
    vel = cboid.getvelocity()
    return unitvector(vel)

def cohesion(M, boid):
    vec = np.array([0., 0.])
    for other in M:
        vec += other.getposition()
    vec /= len(M)
    vec -= boid.getposition()
    if np.isclose(np.linalg.norm(vec), 0):
        return np.array([0., 0.])
    return unitvector(vec)

def avoidance(A, boid):
    mindist = np.Inf
    closestobstacle = Obstacle()
    for obj in A:
        d = boid.distancetoother(obj)
        if d < mindist and d != 0:
            mindist = d
            closestobstacle = obj
    v = -boid.getposition() + closestobstacle.getposition()
    if np.linalg.norm(v) <= DAVOIDANCE:
        return unitvector(v)
    return np.zeros(2)

def attractance(AttractorSet,boid):
    visibleAttractors = set()
    for attractor in AttractorSet:
        if boid.othervisible(attractor) and attractor.isactiveforboid(boid):
          visibleAttractors.add(attractor)

    mindist = np.Inf
    closestattractor = Attractor()
    for obj in visibleAttractors:   #Find the closest attractor
        d = boid.distancetoother(obj)
        if d < mindist and d != 0:
            mindist = d
            closestattractor = obj

    if(mindist <= DATTRACTANCE): ## If the boid is within the radius, the boid visits the attractor
      print("Boid:",boid.getid(),"visited attractor")
      closestattractor.visit(boid)
      boid.setposition(closestattractor.getposition())
    v = closestattractor.getposition() - boid.getposition()
    for attractor in AttractorSet:
      attractor.update()
    if np.linalg.norm(v) < DATTRACTANCE:
        return unitvector(v)
    return np.zeros(2)



In [ ]:
def flock(B, ObstacleSet,AttractorSet, ws=0.0, wa=0.0, wc=0.0, wf=0.2, wattr=0.2, maxspeed=5):
    F = setdeepcopy(B)
    R = set()
    while len(F) > 0:
        f = F.pop()
        C = setcombine(F, R)
        V = set()
        for boid in C:
            if f.othervisible(boid):
                V.add(boid.copy())
        a = np.zeros(2)
        print("boid position",f.getposition())
        print("boid velocity",f.getvelocity())
        print("boid heading",f.getheading())
        if len(V) == 0 or f.getleader():
            f.move()
        else:
            a += ws * separation(V, f)
            a += wa * alignment(V, f)
            a += wc * cohesion(V, f)
        visibleObstacles = set()
        for object in ObstacleSet:
            if f.othervisible(object):
                visibleObstacles.add(object)
        #a += wf * avoidance(visibleObstacles, f)

        attr_acc = attractance(AttractorSet,f)
        print("attracrance",attr_acc)
        a += wattr * attr_acc


        b = f.copy()
        b.addvelocity(a)
        if np.linalg.norm(b.getvelocity()) > maxspeed:
            b.setvelocity(maxspeed)
        b.move()
        R.update({b})
    return R

In [ ]:
fig, ax = plt.subplots()
positionsObstacle = [np.array([-.5, 5.]), np.array([.5, 3]), np.array([2.,2.]), np.array([4.,4.]), np.array([6.,4.])]
radiiObstacle = [0.1, 0.1, 0.1, 0.1, 0.1]
positionsAttractor = [np.array([-.5, 4.]), np.array([.5, 2]), np.array([2.,1.]), np.array([4.,3.]), np.array([6.,3.])]
radiiAttractor = [0.1, 0.1, 0.1, 0.1, 0.1]
obsset = obstacleset(positionsObstacle, radiiObstacle)
aset = attractorset(positionsAttractor, radiiAttractor)
F = flockset(13, [0, 1], [0, 1], 0.1, 1., obsset, 12)
fig,ax = plotobstacleset(obsset,fig,ax)
fig,ax=plotattractorset(aset,fig,ax)
fig,ax=plotflockset(F, fig,ax)
plt.show()

In [ ]:
"""for ind in range(10):
    plt.figure()
    F = flock(F, obsset,aset)
    fig,ax = plotobstacleset(obsset)
    fig,ax = plotflockset(F, fig,ax)
    plotattractorset(aset,fig,ax)

    fig.tight_layout()
    plt.show()"""

fig,ax = plt.subplots()
def update(frame):
    global F,ax,fig,obsset,aset
    if frame == 0:
      #positionsObstacle = [np.array([-.5, 5.]), np.array([.5, 3]), np.array([2.,2.]), np.array([4.,4.]), np.array([6.,4.])]
      #radiiObstacle = [0.1, 0.1, 0.1, 0.1, 0.1]
      #positionsAttractor = [np.array([-.5, 4.]), np.array([.5, 2]), np.array([2.,1.]), np.array([4.,3.]), np.array([6.,3.])]
      #radiiAttractor = [0.1, 0.1, 0.1, 0.1, 0.1]
      positionsObstacle = [np.array([-.5, 5.])]
      radiiObstacle = [0.1]
      positionsAttractor = [np.array([1.1, 1.1])]
      radiiAttractor = [0.1]
      obsset = obstacleset(positionsObstacle, radiiObstacle)
      aset = attractorset(positionsAttractor, radiiAttractor)
      F = flockset(1, [0, 1], [0, 1], 0.1, 1., obsset, 0)

    ax.clear()
    fig,ax = plotobstacleset(obsset,fig,ax)
    fig,ax = plotflockset(F, fig,ax)
    fig,ax = plotattractorset(aset,fig,ax)
    F = flock(F, obsset,aset)


    fig.tight_layout()




ani = animation.FuncAnimation(fig, update, frames=20, interval=1000, blit=False)
#ani.save('set.gif', writer='pillow')
HTML(ani.to_html5_video())


In [ ]:
positionsAttractor = [np.array([-.5, 4.]), np.array([.5, 2]), np.array([2.,1.]), np.array([4.,3.]), np.array([6.,3.])]
radiiAttractor = [0.1, 0.1, 0.1, 0.1, 0.1]
aset = attractorset(positionsAttractor, radiiAttractor)

newSet = set()
for attr in aset:
  print("aset",attr.getposition())
  if np.linalg.norm(attr.getposition() - [4,4]) < 2:
    x = attr
    newSet.add(attr)


def change(newSet):
  for object in newSet:
    print("newSet",object.getposition())
    object.setposition([11.,11.])
  def changeagain(setx):
    for object in setx:
      print("changeagain",object.getposition())
      object.setposition([13.,13.])
  changeagain(newSet)

change(newSet)
for object in aset:
  print("aset",object.getposition())

x.setposition([17.,17.])
for object in aset:
  print("aset x",object.getposition())
